In [1]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

Active code page: 936


ERROR: Invalid requirement: '#'


### ----開始執行----

In [2]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/17 23:29:11
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [3]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

### ----更改股票代號----

In [29]:
stock_id = "1218" #1605 1218 6446
url = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=" + stock_id
#payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [30]:
url = "https://goodinfo.tw/tw/ShowK_ChartFlow.asp?RPT_CAT=DR_3M&STOCK_ID="+stock_id+"&CHT_CAT=MONTH"
#,data=payloads
resp = requests.request(method="POST",url=url,headers=headers)
resp.encoding = "utf-8"

flow_table = pd.read_html(resp.text)

In [31]:
df = flow_table[17].dropna(axis='rows')
#df.drop("季均線乖離率換算價格",axis=1)
df.columns = df.columns.droplevel()
df = df.drop(['-40%','-30%','-20%','-10%','40%','30%','20%','10%'],axis=1)
df
'''   0       1       2       3      4       5      '''  

'   0       1       2       3      4       5      '

In [32]:
products_list = df.values.tolist()
print(products_list[78])
del products_list[119]
del products_list[118]
del products_list[99]
del products_list[98]
del products_list[79]
del products_list[78]
del products_list[59]
del products_list[58]
del products_list[39]
del products_list[38]
del products_list[19]
del products_list[18]
lists_num=len(products_list)
print(len(products_list))
print(products_list)


['交易月份', '收盤價', '漲跌價', '漲跌幅', '目前乖離(%)', '季均線乖離率換算價格']
120
[['22M12', '36.5', '-10.5', '-22.3%', '-10.6', '40.82'], ['22M11', '47', '+8.05', '+20.7%', '+16', '40.5'], ['22M10', '38.95', '+3.4', '+9.56%', '+7.2', '36.33'], ['22M09', '35.55', '+1.05', '+3.04%', '+1.96', '34.87'], ['22M08', '34.5', '-0.05', '-0.14%', '+3.97', '33.18'], ['22M07', '34.55', '+4.05', '+13.3%', '+7.24', '32.22'], ['22M06', '30.5', '-1.1', '-3.48%', '-2.24', '31.2'], ['22M05', '31.6', '+0.1', '+0.32%', '+4.06', '30.37'], ['22M04', '31.5', '+3.5', '+12.5%', '+8.12', '29.13'], ['22M03', '28', '+0.1', '+0.36%', '+1.57', '27.57'], ['22M02', '27.9', '+1.1', '+4.1%', '+2.07', '27.33'], ['22M01', '26.8', '-0.5', '-1.83%', '-0.56', '26.95'], ['21M12', '27.3', '+0.55', '+2.06%', '+0.68', '27.12'], ['21M11', '26.75', '-0.55', '-2.01%', '-0.99', '27.02'], ['21M10', '27.3', '+0.3', '+1.11%', '-1.74', '27.78'], ['21M09', '27', '-2.05', '-7.06%', '-4.82', '28.37'], ['21M08', '29.05', '0', '0%', '-0.51', '29.2'], ['21M07', '2

In [33]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


### 創表

In [16]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令


sqlStr ='''create table 股票各月乖離與均線表
(
	股票各月乖離與均線表編號 nvarchar(20) primary key
	,收盤價 nvarchar(20) NULL
	,漲跌價 nvarchar(20) Null
	,[目前乖離(%)] nvarchar(20) Null
	,均線 nvarchar(20) Null
	,交易月份 nvarchar(5) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

### 資料入庫

In [34]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# R+代號+發月別
# MYSQL指令
for i in range(0,lists_num):
    Rstock_id='R'+str(stock_id)+str(products_list[i][0])
    sqlStr = f""" 
    Insert into 股票各月乖離與均線表 ([股票各月乖離與均線表編號], 
    [收盤價], [漲跌價],
    [目前乖離(%)],[均線],
    交易月份,股票代號)
    values ('{str(Rstock_id)}',
    '{products_list[i][1]}','{products_list[i][2]}','{products_list[i][4]}','{products_list[i][5]}',
    '{products_list[i][0]}','{stock_id}')
    """
    cursor.execute(sqlStr)

print(sqlStr)

 
    Insert into 股票各月乖離與均線表 ([股票各月乖離與均線表編號], 
    [收盤價], [漲跌價],
    [目前乖離(%)],[均線],
    交易月份,股票代號)
    values ('R121813M01',
    '15.75','-0.2','-0.32','15.8',
    '13M01','1218')
    


In [ ]:
'''
R+代號+發月別
交易月份	收盤價	漲跌價		目前乖離(%)	均線'''